# md5cypto
Voy a reescribir el algoritomo en python para poder obtener un mensaje de prueba donde conzca todas las variables.

## Librerias y recursos que me sirvieron
- https://docs.python.org/3/library/hashlib.html
- https://docs.python.org/3/library/base64.html#base64.b64decode

In [1]:
import hashlib
import base64
import requests
response = requests.get(url="https://ciberseguridad.diplomatura.unc.edu.ar/cripto/md5crypt/alvarmaciel@gmail.com/challenge")
encoded_text = response.text

# El plan es generar un texto encodeado con una version de la librería y decodearlo
plaintext = b"Hola tu@email.com"
key = b"12345"
# No se si hacerlo en bytes o en hex, creo que es lo mesmo. Lo hago en ambos para asegurarme


In [60]:
def encrypt(plaintext:bytes) -> bytearray: #Saco la key para pasarla en un segundo momento
    encrypt_key_plain = b"1234" # Por ahora uso un valor definido para probar. EL nombre es horrible
    encrypt_key_hex = hashlib.md5(encrypt_key_plain).hexdigest()
    primer_pasada=bytearray()
    for i, c in enumerate(plaintext): # uso enumerate para generar el contador
        counter = encrypt_key_hex[i % len(encrypt_key_hex)] # contador vuelve a 0 con el largo de la clave
        primer_pasada.append(ord(counter))
        primer_pasada.append(ord(counter) ^ c)
    return primer_pasada


In [61]:
# segunda pasada, acá key es la clave externa
def key_ed(primer_pasada: bytes, key: bytes) -> bytearray: 
    key_md5 = hashlib.md5(key).hexdigest()
    encrypted = bytearray()

    for i, b in enumerate(primer_pasada):
        encrypted.append(b ^ ord(key_md5[i % len(key_md5)]))
    return base64.b64encode(encrypted).decode()


In [62]:
primer_pasada = encrypt(plaintext)
# Chequeo que sea el doble
assert len(primer_pasada) == len(plaintext)*2
print(primer_pasada)
bytearray.hex(primer_pasada)

bytearray(b'8p1^d\x08c\x029\x19b\x16d\x11b"5P2_d\x050Y4XdJc\x002]0]')


'3870315e6408630239196216641162223550325f640530593458644a6300325d305d'

In [63]:
# segunda pasada el xor con la clave externa
encrypted_message = key_ed(primer_pasada, key)
print(bytes.hex(base64.b64decode(encrypted_message)))
print(base64.b64decode(encrypted_message))
print(encrypted_message)

0042063d076a53675c785a77532154410133016b053406610d6902725765053f086f
b'\x00B\x06=\x07jSg\\xZwS!TA\x013\x01k\x054\x06a\ri\x02rWe\x05?\x08o'
AEIGPQdqU2dceFp3UyFUQQEzAWsFNAZhDWkCcldlBT8Ibw==


In [2]:
# Remove internal encrypted_key
# byte_message = encrypted_message.encode()
b64_decoded_message = base64.b64decode(encoded_text)
pares = b64_decoded_message[0::2]
impares = b64_decoded_message[1::2]
remove_internal_key = bytearray(par ^ impar for par, impar in zip(pares, impares))
print(len(remove_internal_key)) # Tiene que ser 17
print(bytearray.hex(remove_internal_key))

358
00656a21376225302369626639626c17336731683a20303e3c0e45773f6a3a7701793573763226223476436028666d27386f7e6239636d5b126b6d713569747a207320646c2e273429702c753c666939782a3369377c203425397671362a385d106b24646c2e04343528233465274a223a2a6231643c7363623e30356a3239777f3a603166040024336e6666243d200531307050236127345b47603f70527332262a6c74256b2111347c6268206b657937653d3f7a2e39353e61436028666d27386f7e623963595b7f2a2d25276f6539746c3975252e3c3771677160317369213d7e2921247b3d712270716a3e602c7739652264767a3b303f246c6b352770253b6d22603b633623716b71252775692331785a69377d733334616d253b696f203a2a246e766f31303f606c6b70736832746e35723d7a3c2171626c777073683274673f73332e202130676a6a2574203138653f7378045a587c2923432262647716783f6e3d7d


In [3]:

key_candidates = set(range(48, 58)) | set(range(97, 103))
texto_conocido= "alvarmaciel@gmail.com"
largo = len(texto_conocido)
primer_letra = texto_conocido[0]


In [15]:
def buscar_offsets_candidatos(remove_internal_key, texto_conocdido):
    valores_validos = [ord(c) for c in "0123456789abcdef"]
    deltas_validos = [a ^ b for a in valores_validos for b in valores_validos]
    periodo = 16
    
    candidatos = []
    
    for offset in range(periodo):
        deltas = [None] * periodo
        valido = True

        # loop para cada letra del texto conocido
        for j, ch in enumerate(texto_conocdido):
            i = offset + j
            if i >= len(remove_internal_key):
                break

            p = ord(ch) # byte claro esperado
            d = remove_internal_key[i] ^ p
            k = i % periodo

            # Validadción
            if d not in deltas_validos:
                valido = False
                break # me muevo al siguiente valor dentro dle periodo
            if deltas[k] is not None and deltas[k] != d:
                valido=False
                break
            deltas[k] = d

        if valido:
            candidatos.append((offset, deltas))
    return candidatos


In [17]:
decode_offset_deltas = buscar_offsets_candidatos(remove_internal_key, texto_conocido)
decode_offset_deltas

[(4, [84, 10, 80, 1, 86, 14, 83, 81, 81, 4, 3, 5, 80, 7, 0, 87])]

In [21]:
def recover_paintext_partial(remove_internal_key, deltas):
    out = bytearray()
    for i, r in enumerate(recover_paintext_partial):
        d = deltas[i % 16]
        if d is None:
            out.append(0x3f)
        else:
            out.append(r ^ d)
    return out

In [27]:
deltas = decode_offset_deltas[0][1]
print(recover_paintext_partial(remove_internal_key, deltas).decode('utf-8'))

To: alvarmaciel@gmail.com
From: User <user@example.com>
Content-type: text/plain, charset=utf-8
Date: Wed, 15 Jun 2022 23:30:59 +0000
Subject: Re: Quote
Cc: User <user@example.com>, jdoe@example.com

... when fits of creativity run strong, more than one programmer or writer
has been known to abandon the desktop for the more spacious floor.
		-- Fred Brooks


In [89]:
# url = "https://ciberseguridad.diplomatura.unc.edu.ar/cripto/md5crypt/alvarmaciel@gmail.com/answer"

# with open('message.txt', 'rb') as archivo:
#     files = {'message': ('message.txt', archivo, 'text/plain')}
#     response = requests.post(url, files=files)

# # Verificar la respuesta
# print(f"Código de estado: {response.status_code}")
# print(f"Respuesta: {response.text}")

Código de estado: 200
Respuesta: ¡Ganaste!



In [98]:
HEX_ASCII = [ord(c) for c in "0123456789abcdef"]
valores_validos = set(range(48, 58)) | set(range(97, 103))
print(HEX_ASCII)

[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 97, 98, 99, 100, 101, 102]


In [99]:
print(valores_validos)

{97, 98, 99, 100, 101, 102, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57}
